In [1]:
import pandas as pd
import xml.etree.ElementTree as ET
import glob
import os


_dir='C:/Users/ccham/Desktop/est/est'

os.chdir(_dir)

elements = ['Image','height','width','depth','coin_1','coin_2','coin_3','coin_4']

df=pd.DataFrame(columns=elements)

# Initialize empty lists to store coin information
coin_names = []
coin_values = {}


# Iterate over XML files in the directory
for xml_file in glob.glob('*.xml'):
    if xml_file.endswith('.xml'):
        # Parse the XML file
        tree = ET.parse(os.path.join(_dir, xml_file))
        root = tree.getroot()

        # Extract the annotation file name
        file_name = root.find('filename').text
        
        # Extract size information
        width = int(root.find('size/width').text)
        height = int(root.find('size/height').text)
        depth = int(root.find('size/depth').text)
        
        
        # Iterate over the 'object' elements
        for obj in root.iter('object'):
            # Extract the coin name
            coin_name = obj.find('name').text
            
            # Extract the coin value (xmin, ymin, xmax, ymax)
            bbox = obj.find('bndbox')
            xmin = int(bbox.find('xmin').text)
            ymin = int(bbox.find('ymin').text)
            xmax = int(bbox.find('xmax').text)
            ymax = int(bbox.find('ymax').text)

            # Store the coin value in the dictionary
            coin_values[coin_name] = (xmin, xmax, ymin, ymax)
    
        tmp_df=pd.DataFrame([coin_values], columns=['Image','height','width','depth'] + list(coin_values.keys()))
        # Set the 'filename' column value
        tmp_df['Image'] = file_name
        tmp_df['height']=height
        tmp_df['width']=width
        tmp_df['depth']=depth
        # Append the data to the DataFrame
        df = pd.concat([df,tmp_df], ignore_index=True)
        


# Print the DataFrame
print(df.describe)

<bound method NDFrame.describe of      Image height width depth                coin_1                coin_2  \
0    0.jpg    448   448     3      (77, 77, 73, 73)    (17, 17, 245, 245)   
1    1.jpg    448   448     3    (89, 89, 167, 167)      (93, 93, 93, 93)   
2   10.jpg    448   448     3    (78, 78, 341, 341)    (20, 20, 186, 186)   
3   11.jpg    448   448     3    (69, 69, 340, 340)    (31, 31, 213, 213)   
4   12.jpg    448   448     3    (72, 72, 338, 338)    (32, 32, 209, 209)   
5   13.jpg    448   448     3    (74, 74, 277, 277)    (26, 26, 163, 163)   
6   14.jpg    448   448     3  (100, 100, 284, 284)    (42, 42, 169, 169)   
7   15.jpg    448   448     3    (87, 87, 256, 256)    (35, 35, 158, 158)   
8   16.jpg    448   448     3  (101, 101, 249, 249)    (39, 39, 147, 147)   
9   17.jpg    448   448     3    (66, 66, 262, 262)    (19, 19, 171, 171)   
10  18.jpg    448   448     3    (81, 81, 287, 287)    (28, 28, 196, 196)   
11  19.jpg    448   448     3    (72, 72, 

In [2]:
print([df[df[f'coin_{i}'].apply(lambda x: x[2]) != df[f'coin_{i}'].apply(lambda x: x[3])] for i in range(1,5)])
print(df[df.height<448])

[Empty DataFrame
Columns: [Image, height, width, depth, coin_1, coin_2, coin_3, coin_4]
Index: [], Empty DataFrame
Columns: [Image, height, width, depth, coin_1, coin_2, coin_3, coin_4]
Index: [], Empty DataFrame
Columns: [Image, height, width, depth, coin_1, coin_2, coin_3, coin_4]
Index: [], Empty DataFrame
Columns: [Image, height, width, depth, coin_1, coin_2, coin_3, coin_4]
Index: []]
Empty DataFrame
Columns: [Image, height, width, depth, coin_1, coin_2, coin_3, coin_4]
Index: []


In [8]:
!pip install opencv-python

In [3]:
import matplotlib.pyplot as plt
import cv2

images=df.Image

# Visualize images and their corresponding angle points
for i in range(len(images)):
    image = cv2.imread(images[i])
    angle = [df.loc[i,f'coin_{x}'] for x in range(1,5)]

    # Plot image with angle points
    plt.figure()
    plt.imshow(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    for p in angle:
        plt.scatter([p[0],p[1]], [p[2],p[3]], color='red')  # Plot the x and y coordinates separately
        # Draw a line between the two points
        plt.plot()
    # Draw lines between the angle points
    for j in [0,2]:
        x_coords = [angle[j][0], angle[j + 1][0]]  # x-coordinates
        y_coords = [angle[j][2], angle[j + 1][2]]  # y-coordinates
        x_c00rds = [angle[j][0],angle[j][0]]
        y_c00rds = [448,0]
        plt.plot(x_coords, y_coords, color='blue')
        plt.plot(x_c00rds, y_c00rds, color='green')
    plt.title('Teeth Image with Angle Points')
    plt.show()

ModuleNotFoundError: No module named 'cv2'